
# LangGraph with Vector Search Tool - RAG-Enabled AI Agent

This code creates a **Retrieval-Augmented Generation (RAG)** system using LangGraph + Databricks Vector Search for intelligent document retrieval.

## 🔧 **Setup & Components**
- **LLM**: Databricks GPT OSS 120B model
- **Vector Search**: Databricks Vector Search as a retrieval tool
- **Graph**: LangGraph for orchestrating LLM + retrieval workflow

## 🔍 **Vector Search Tool Configuration**
```python
vs_tool = VectorSearchRetrieverTool(
  index_name="agents.main.foodly_policy_embedding_index",
  tool_name="foodly_policy_document_retrieval_tool", 
  num_results=2,
  tool_description="Search Foodly knowledge base for policies, procedures..."
)
```
- **Index**: Points to a pre-built vector index containing Foodly company documents
- **Retrieval**: Returns top 2 most relevant document chunks
- **Scope**: Searches policies, refund rules, delivery guidelines, etc.
- **Smart description**: Helps LLM understand when to use this tool

## 🤖 **LLM + Tool Integration**
```python
llm_with_tools = llm.bind_tools([vs_tool])  # LLM can now search documents
```
The LLM gains the ability to search through company documentation when needed.

## 🕸️ **Graph Architecture**


- **llm node**: Processes user queries and decides if document search is needed
- **tools node**: Executes vector search to retrieve relevant documents  
- **Conditional routing**: Automatically searches docs when LLM determines it's necessary

## 🔄 **RAG Workflow**
1. **User question**: e.g., *"What's the refund policy?"*
2. **LLM analysis**: Determines this needs company policy information
3. **Vector search**: Retrieves relevant policy documents from the index
4. **Augmented response**: LLM answers using retrieved company documents
5. **Accurate answer**: Response based on actual company policies, not general knowledge

## 🎯 **Key Benefits**
- **Up-to-date info**: Always uses current company documents
- **Accurate responses**: Grounded in actual company policies
- **Automatic retrieval**: LLM decides when to search documents
- **Scalable**: Works with large document collections
- **Enterprise-ready**: Uses Databricks' managed vector search

This pattern enables AI assistants to provide accurate, company-specific answers by automatically retrieving relevant documentation when needed.


In [0]:
!pip install -q langchain==0.3.14
!pip install -q langchain-openai==0.3.0
!pip install -q langchain-community==0.3.14
!pip install -q langgraph==0.2.64

In [0]:
%pip install -U -qqqq databricks-langchain uv databricks-agents mlflow-skinny[databricks]

In [0]:
dbutils.library.restartPython()

In [0]:
from dotenv import load_dotenv
import os
from typing import Literal, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt.tool_node import ToolNode, tools_condition



from databricks_langchain import (
    ChatDatabricks,
    VectorSearchRetrieverTool
)



# Initialize LLM
llm = ChatDatabricks(endpoint="databricks-gpt-oss-120b")



def call_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}


# Initialize the retriever tool.
vs_tool = VectorSearchRetrieverTool(
  index_name="agentic_ai.langgraph.router_agent_index",
  tool_name="customer_support_retrieval_tool",
  num_results=2,
  tool_description="Use this tool to search the Customer Support Document for customer questions and category of the question. It retrieves the most relevant chunks from the company’s customer support documentation, including customer questions, and category of the question"
)

In [0]:
query = 'what is your refund policy?'
metadata_filter = {'category' : 'general'}
# Update retriever search_kwargs dynamically
# vs_tool.search_kwargs["filter"] = metadata_filter
vs_tool.invoke(query)

In [0]:
result = vs_tool.invoke({"query": "what is your refund policy?"})

In [0]:
for elem in result:
    print(type(elem))

In [0]:
llm_with_tools = llm.bind_tools([vs_tool])

builder = StateGraph(MessagesState)

builder.add_node("llm",call_llm)
builder.add_node("tools",ToolNode([vs_tool]))


builder.add_edge(START,"llm")
builder.add_conditional_edges("llm" , tools_condition)
builder.add_edge("tools","llm")


agent = builder.compile()



In [0]:
from IPython.display import display, Image

Image(agent.get_graph().draw_mermaid_png())

In [0]:
messages = agent.invoke({"messages": [HumanMessage("what is your refund policy? I am really fed up with this product and need to refund it?")]})

last_message = messages["messages"][-1].content
print(last_message)

In [0]:
messages = agent.invoke({"messages": [HumanMessage("what is your refund policy? I am really fed up with this product and need to refund it?")]})

last_message = messages["messages"][-1].content


import markdown

# Convert markdown → HTML
html = markdown.markdown(last_message, extensions=["tables", "fenced_code"] )

# Render in the notebook automatically
displayHTML(html)

In [0]:
messages = agent.invoke({"messages": [HumanMessage("I bought it like 10 days ago , can i make the refund?")]})

last_message = messages["messages"][-1].content
print(last_message)